In [8]:
import ee
import pandas as pd

ee.Initialize()

# 관심 지역 및 기간 설정
aoi = ee.Geometry.Polygon(
    [[[
        128.852806, 35.003003
    ], [
        128.852806, 35.044895
    ], [
        128.940868, 35.044895
    ], [
        128.940868, 35.003003
    ], [
        128.852806, 35.003003
    ]]]
)
start_date = '2021-01-01'
end_date = '2023-10-31'

# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    red = image.select('B4')   # Red band
    nir = image.select('B8')   # NIR band
    swir1 = image.select('B11') # SWIR1 band

    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# NDWI가 0.5 이상이고 FAI가 50 이상인 지역의 면적 계산
def calculate_area(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mask = image.select('NDWI').gt(0.1).And(image.select('FAI').gt(-50))
    area = mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10  # Sentinel-2 픽셀 해상도
    ).get('NDWI')
    return ee.Feature(None, {'date': date, 'area': area})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용 후 면적 계산
processed_collection = sentinel2.map(calculate_indices)
area_series = processed_collection.map(calculate_area)

# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
area_features = area_series.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in area_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')


In [9]:
# Calculate the area of the AOI in square meters.
area = aoi.area().getInfo()

# Convert the area to square kilometers (1 square kilometer = 1,000,000 square meters).
area_sq_km = area / 1_000_000

print("Area of the AOI in square kilometers:", area_sq_km)

Area of the AOI in square kilometers: 37.353208256071845


In [11]:
df

,area,date
0,42638.362166,2021-01-04
1,282436.305985,2021-01-09
2,81865.357758,2021-01-14
3,232386.609548,2021-01-19
4,123798.758484,2021-01-29
...,...,...
82,56893.975802,2023-09-06
83,2602.076248,2023-10-01
84,21204.781332,2023-10-11
85,900.717812,2023-10-16


In [10]:
# Sentinel-2 ImageCollection 필터링
sentinel2_image = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .first()

In [14]:
import folium

# Sentinel-2 이미지 컬렉션에 NDWI와 FAI 지수 계산 적용
processed_image = sentinel2.map(calculate_indices).first()

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium Map 클래스에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 시각화 매개변수 설정
ndwi_vis_params = {'min': 0.0, 'max': 1.0, 'palette': ['00FFFF', '0000FF']}  # Cyan to Blue
fai_vis_params = {'min': -50, 'max': 50, 'palette': ['FF0000', '00FF00']}  # Red to Green

# 이미지의 NDWI 및 FAI 밴드 선택
ndwi_layer = processed_image.select('NDWI')
fai_layer = processed_image.select('FAI')

# Folium 지도 객체 생성
map = folium.Map(location=[35.023949, 128.896837], zoom_start=12)

# NDWI 레이어 추가
map.add_ee_layer(ndwi_layer, ndwi_vis_params, 'NDWI')

# FAI 레이어 추가
map.add_ee_layer(fai_layer, fai_vis_params, 'FAI')

# 맵 표시
map.add_child(folium.LayerControl())
map



AttributeError: 'ImageCollection' object has no attribute 'second'